<a href="https://colab.research.google.com/github/eip4-mars/EIP4P2/blob/master/Session2/Initial_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchsummary import summary

use_cuda= torch.cuda.is_available()
device=torch.device('cuda' if use_cuda else 'cpu')

model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, 4)
model = nn.Sequential(model, nn.LogSoftmax())
model = model.to(device)
summary(model, input_size=(3,224,224))

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/checkpoints/mobilenet_v2-b0353104.pth



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount =True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
'''
##It takes time Dont Run
import shutil, os
from tqdm import tqdm

os.mkdir('Session2_Dataset')
src = '/content/drive/My Drive/Session 2 Dataset/'
dst = '/content/Session2_Dataset/'

for cls in os.listdir(src):
    if not os.path.exists(dst+cls):
        os.makedirs(dst+cls)
    
    for file in tqdm(os.listdir(src+cls)):
        shutil.copy(src+cls+'/'+file, dst+cls)
'''

In [3]:
!unzip /content/drive/My\ Drive/Session\ 2\ Dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: Session 2 Dataset/Small QuadCopters/modern-cameras-quadcopter-modern-cameras-quadcopter-flies-against-blue-sky-shoots-video-103702954.jpg  
  inflating: Session 2 Dataset/Small QuadCopters/modern-drone-rushing-to-explore-beautiful-iris-stretching-over-hills-sunny-day-small-rain-contemporary-white-101067098.jpg  
  inflating: Session 2 Dataset/Small QuadCopters/modern-rc-drone-quadcopter-camera-flying-108221606.jpg  
  inflating: Session 2 Dataset/Small QuadCopters/modern-rc-drone-quadcopter-camera-flying-108364403.jpg  
  inflating: Session 2 Dataset/Small QuadCopters/modern-rc-drone-quadcopter-camera-flying-mountain-vysoke-tatry-slovakia-small-dron-landing-trail-modern-rc-125085424.jpg  
  inflating: Session 2 Dataset/Small QuadCopters/modern-rc-drone-quadcopter-camera-flying-sunrise-141052834.jpg  
  inflating: Session 2 Dataset/Small QuadCopters/modern-rc-drone-quadcopter-camera-flying-sunrise-141053322.jpg  
  inflatin

In [ ]:
##rm -rf Session2_Dataset/

In [4]:
!du -sh Session\ 2\ Dataset/

1.6G	Session 2 Dataset/


In [ ]:
!ls -Slhr Session\ 2\ Dataset/Small\ QuadCopters | wc -l

3292


In [ ]:
!ls -Slhr Session2_Dataset/Large\ QuadCopters | tail -200 |head -1 

-rw------- 1 root root 7.4K Jul 24 07:42 photo-1484666086787-6cd0ccd39860.jpg


In [ ]:
!ls -Slhr Session2_Dataset/Small\ QuadCopters | tail -200 |head -1 

-rw------- 1 root root  90K Jul 23 15:58 abstract-conceptual-child-playing-drone-remote-sensing-la-felicità-di-un-bambino-con-giocattolo-moderno-e-pericoloso-107771953.jpg


In [ ]:
!ls -Slhr Session2_Dataset/Winged\ Drones | tail -200 |head -1 

-rw------- 1 root root  672K Jul 23 15:18 wing-testers-receiving-package-as-part-of-faa-testing_wide-555d7c5d81e7172b02979c27f18b25585106abbd.jpg


In [5]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm
import traceback
import warnings

warnings.filterwarnings("error")

path = "/content/Session 2 Dataset/"
classes = os.listdir(path)
train = []
test = []
labels = pd.DataFrame(classes,index=range(len(classes)), columns=['Label'])
labels.reset_index(inplace=True)


test_ratio = 0.2

#print(labels)
for i in range(len(classes)):
    child_path = path+classes[i]+'/'
    imgs = os.listdir(child_path)
    #print(len(tr),classes[i])
    trn,tst = train_test_split(imgs, test_size=test_ratio, random_state=42)
    
    train += [[child_path+x, i] for x in trn]
    test += [[child_path+x, i] for x in tst]

# train1 = train.copy()
# test1 = test.copy()

for impath in tqdm(train):
    try:
        im = Image.open(impath[0])
        im.close()
    except:
        train.remove(impath)
        pass

for impath in tqdm(test):
    try:
        im = Image.open(impath[0])
        im.close()
    except:
        test.remove(impath)
        pass

train_df = pd.DataFrame(train,columns=['File_name', 'label_index'])
test_df = pd.DataFrame(test,columns=['File_name', 'label_index'])

print(train_df.shape, test_df.shape, labels.shape)
#train_df_1 = pd.concat([train_df[["File_name"]], pd.get_dummies(train_df.label_index, prefix="label")], axis = 1)
#test_df_1 = pd.concat([test_df[["File_name"]], pd.get_dummies(test_df.label_index, prefix="label")], axis = 1)

train_df.to_csv('train.csv',index=False)
test_df.to_csv('test.csv',index=False)
labels.to_csv('labels.csv',index=False)
warnings.filterwarnings("ignore")

100%|██████████| 4148/4148 [00:00<00:00, 6674.53it/s]


(16582, 2) (4148, 2) (4, 2)


In [6]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
import random

class custDataset(Dataset):
	"""Face Landmarks dataset."""

	def __init__(self, csv_file, transform=None):
		self.frame = pd.read_csv(csv_file, header=0)
		self.transform = transform

	def __getitem__(self, idx):
		image_name = self.frame.iloc[idx, 0]
		label = self.frame.iloc[idx, 1]
		
		image = Image.open(image_name).convert('RGB')
		if self.transform:
			image = self.transform(image)
		#sample = {'image': image, 'label': label}
		return image, label
		
	def __len__(self):
		return len(self.frame)

def getTrainData(batch_size=64):
	
	stats = {'mean': [0.3931, 0.3785, 0.3606],
			 'std': [0.1965, 0.1813, 0.1779]}

	transformed_training = custDataset(csv_file='/content/train.csv',
											transform=transforms.Compose([
											transforms.Resize(224),
											transforms.CenterCrop(224),
											transforms.RandomHorizontalFlip(),
											transforms.RandomRotation(10),
											transforms.ToTensor(),
											transforms.Normalize(stats['mean'],stats['std'])
										]))

	dataloader_training = DataLoader(transformed_training, batch_size,
									 shuffle=True, num_workers=4, pin_memory=False)

	return dataloader_training



def getTestData(batch_size=64):

	stats = {'mean': [0.3931, 0.3785, 0.3606],
			 'std': [0.1965, 0.1813, 0.1779]}
	
	transformed_testing = custDataset(csv_file='/content/test.csv',
										transform=transforms.Compose([
										transforms.Resize(224),
										transforms.CenterCrop(224),
										transforms.ToTensor(),
										transforms.Normalize(stats['mean'],stats['std'])
									]))

	dataloader_testing = DataLoader(transformed_testing, batch_size,
									shuffle=True, num_workers=4, pin_memory=False)

	return dataloader_testing


In [7]:
train_loader = getTrainData(64)
test_loader = getTestData(64)

In [8]:
!git clone https://github.com/eip4-mars/eva4_lib.git

Cloning into 'eva4_lib'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 22 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [9]:
from eva4_lib.utils.train_test import train , test, predict, get_misclassified
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

optimizer = optim.SGD(model.parameters(), lr=0.01,  momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=1e-06, eps=1e-06)

for epoch in range(1, 3):
    curr_lr=optimizer.param_groups[0]['lr']
    print(f'Epoch: {epoch} Learning_Rate {curr_lr}')
    train(model, device, train_loader, optimizer, epoch)
    test_acc1 = test(model, device, test_loader)
    print('Test acc:', test_acc1)
    scheduler.step(test_acc1)

  0%|          | 0/260 [00:00<?, ?it/s]

Epoch: 1 Learning_Rate 0.01


Loss=1.2337290048599243 Batch_id=259 Accuracy=79.74: 100%|██████████| 260/260 [04:07<00:00,  1.05it/s]
  0%|          | 0/260 [00:00<?, ?it/s]


Test set: Average loss: 0.3924, Accuracy: 3506/4148 (84.52%)

Test acc: 84.52266152362584
Epoch: 2 Learning_Rate 0.01


Loss=0.554164469242096 Batch_id=259 Accuracy=85.23: 100%|██████████| 260/260 [04:12<00:00,  1.03it/s]



Test set: Average loss: 0.3330, Accuracy: 3627/4148 (87.44%)

Test acc: 87.4397299903568


In [16]:
model.to(torch.device('cpu'))
model.eval()
traced_model = torch.jit.trace(model,torch.randn(1,3,244,244))
traced_model.save("Sess2_mobilenetv2_4.pt")

In [ ]:
torch.cuda.empty_cache()